**to do**
- [ ] 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from numba import jit, njit, typeof
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd

In [4]:
from utils.knn.embedding import EmbeddingGaussian

In [4]:
params = {
    'exp_root':'/home/korhan/Desktop/tez/knn_exps/',
    'disc_method': 'knn',
    'feats_root': '/home/korhan/Desktop/tez/dataset/features/',
    'CVroot': '/home/korhan/Dropbox/CVfolds',
    'CVset': 'A',
    'featype': 'c3right',
    'config_file' : '/home/korhan/Dropbox/tez_scripts/config/config_phoenix.json',
    'dataset': 'phoenix',
    'njobs':2,
         }



with open(join(params['CVroot'], params['CVset'] + '.txt'), 'r') as f: 
    seq_names = [x.strip('\n') for x in f.readlines()]
    
print(len(seq_names))

1705


In [6]:
# get feats dict
feats_dict = {}
for name in seq_names: 
    arr = np.load(join(params['feats_root'], 'deep_hand/c3/right/train/', name + '.npy'))
    feats_dict[name] = arr / arr.std(1)[:,None]

# feats_dict = fea_util.apply_PCA_dict(feats_dict, 0.99, whiten=True)
feats_dict[name].shape

(219, 61)

In [7]:
params_emb = {'dim_fix':10,
              'r': 0.07, 
              's':0.1,
              'seg_type': 'uniform'}

params_seg = {'a': 3,
              'lmin' : 5, 
              'lmax' : 25,
              'emb_type': 'gauss_kernel'
             }

params_knn = {'pca': 'PCA40',
              'metric': 'L2',
              'k':100,
              'use_gpu': True}


params_disc = {**params_seg, 
               **params_emb, 
               **params_knn,
              'olapthr_m': 0.25,
              'top_delta' : 0.02,}


params_post = {
                'method': 'pairwise',
                'cost_thr': 0.001}

params['disc'] = params_disc
params['clustering'] = params_post
params['lr'] = 0.8
params['patience'] = 30
params

{'exp_root': '/home/korhan/Desktop/tez/knn_exps/',
 'disc_method': 'knn',
 'feats_root': '/home/korhan/Desktop/tez/dataset/features/',
 'CVroot': '/home/korhan/Dropbox/CVfolds',
 'CVset': 'A',
 'featype': 'c3right',
 'config_file': '/home/korhan/Dropbox/config/config_phoenix.json',
 'dataset': 'phoenix',
 'njobs': 2,
 'disc': {'a': 3,
  'lmin': 5,
  'lmax': 25,
  'emb_type': 'gauss_kernel',
  'dim_fix': 10,
  'r': 0.07,
  's': 0.1,
  'seg_type': 'uniform',
  'pca': 'PCA40',
  'metric': 'L2',
  'k': 100,
  'use_gpu': True,
  'olapthr_m': 0.25,
  'top_delta': 0.02},
 'clustering': {'method': 'pairwise', 'cost_thr': 0.001},
 'lr': 0.8,
 'patience': 30}

In [18]:
from utils.helper_fncs import save_json
save_json(data=params, full_path='/home/korhan/Dropbox/config/knn')

In [9]:
from utils.knn.embedding import EmbeddingGaussian
from utils.knn.faissWrapper import KnnIndex
from utils.knn.segmenter import uniform_segmenter
from utils.knn.pair_select import pair_selection_wrap
import os

def gen_expname(params):
    return '_'.join(['{}_{}'.format(key,v) for key,v in params.items()])


class KnnDiscovery():
    def __init__(self, feats_dict, params):
        self.params = params
        self.index = None
        self.seq_names = list(feats_dict.keys())
        
        if self.params['disc']['seg_type'] == 'uniform':
            # get segmentation intervals per sequence
            self.intervals_dict = uniform_segmenter(feats_dict, 
                                                    self.params['disc'])
            
        if self.params['disc']['emb_type'] == 'gauss_kernel':
            self.embedder = EmbeddingGaussian(**self.params['disc'])
            
        self.run_embed(feats_dict) 
        
            
    def run_embed(self, feats_dict):
        self.X, self.traceback_info = compute_all_embeddings(
            feats_dict, self.intervals_dict, self.embedder)
        
        
    def build_index(self, knn_params):
        
        self.index = KnnIndex(self.X, **knn_params)
        
    
    def run_disc(self, new_disc_params=None):

        if type(new_disc_params) is dict: 
            self.params['disc'] = new_disc_params
            self.build_index(self.params['disc'])
        
        if self.index is None: self.build_index(self.params['disc'])
               
        self.D, self.I = self.index.search(self.X, self.params['disc']['k'])
        self.expname = gen_expname(params['disc'])

        
    def post_disc(self, new_post_params=None):
        if type(new_post_params) is dict: 
            self.params['clustering'] = new_post_params
        
        self.matches_df = pair_selection_wrap(self.D, self.I , 
                                         self.traceback_info, 
                                         self.intervals_dict, 
                                         self.params['clustering'])
        
        self.postdisc_name = gen_expname(params['clustering'])
        
    
    def run(self):
        
        self.run_disc()
        self.post_disc()
        
        return self.matches_df
        
    


In [ ]:
    def evaluate(self ):
        
        nodes_df, clusters_list = match_pairs_as_clusters(self.matches_df)
        self.nodes_df, self.clusters_list = nodes_starts_from1(nodes_df, clusters_list)
        
        self.postdisc_path = os.path.join(self.params['exp_root'], 
                                          self.expname, 
                                          self.postdisc_name)
        
        os.makedirs(self.postdisc_path, exist_ok=True)
        
        pickle_save_nodes_clusters(self.nodes_df, 
                                   self.clusters_list, 
                                   self.postdisc_path)  

        
        self.scores = evaluate_discovery(self.postdisc_path, 
                        jobs=self.params['njobs'], dataset=self.params['dataset'],
                        seq_names=self.seq_names, cnf=self.params['config_file'])
        
        return self.scores

In [1]:
from utils.knn.discoverer import KnnDiscovery

In [8]:
%%time
knndisc = KnnDiscovery(feats_dict, params)

Computing Embeddings
CPU times: user 6.57 s, sys: 11.3 s, total: 17.9 s
Wall time: 18.8 s


In [10]:
%%time
params['disc']['norm'] = False
knndisc.run_disc()

Building index of size 477967x610
Searching index
CPU times: user 19.3 s, sys: 20.8 s, total: 40.1 s
Wall time: 37.4 s


In [13]:
D = knndisc.D
I = knndisc.I

In [14]:
traceback_info, intervals_dict = knndisc.traceback_info, knndisc.intervals_dict

In [46]:
knndisc.matches_df

,seg_id1,seg_id2,f1,f2,f1_id,f2_id,f1_start,f1_end,f2_start,f2_end,cost
0,224,76094,03December_2011_Saturday_tagesschau_default-8,25July_2010_Sunday_tagesschau_default-5,0,247,96,102,168,174,3.374237
1,224,76084,03December_2011_Saturday_tagesschau_default-8,25July_2010_Sunday_tagesschau_default-5,0,247,96,102,159,171,3.525085
2,78,228992,03December_2011_Saturday_tagesschau_default-8,03December_2011_Saturday_tagesschau_default-2,0,750,33,42,63,72,3.946320
3,224,76076,03December_2011_Saturday_tagesschau_default-8,25July_2010_Sunday_tagesschau_default-5,0,247,96,102,156,162,5.057892
4,78,65250,03December_2011_Saturday_tagesschau_default-8,23October_2010_Saturday_tagesschau_default-2,0,212,33,42,114,129,5.109283
5,224,65261,03December_2011_Saturday_tagesschau_default-8,23October_2010_Saturday_tagesschau_default-2,0,212,96,102,120,126,5.325378
6,78,361884,03December_2011_Saturday_tagesschau_default-8,24January_2010_Sunday_tagesschau_default-2,0,1188,33,42,126,135,5.353577
7,84,375961,03December_2011_Saturday_tagesschau_default-8,28December_2011_Wednesday_tagesschau_default-2,0,1233,36,42,18,39,5.408966
8,66,228984,03December_2011_Saturday_tagesschau_default-8,03December_2011_Saturday_tagesschau_default-2,0,750,27,42,60,66,5.546494
9,78,239229,03December_2011_Saturday_tagesschau_default-8,07June_2010_Monday_heute_default-12,0,782,33,42,63,81,5.582062


In [53]:
%%time
params['clustering']['top_delta'] = 0.002
knndisc.post_disc(params['clustering'])

CPU times: user 14.4 s, sys: 7.81 ms, total: 14.4 s
Wall time: 14.4 s


In [48]:
knndisc.matches_df

,seg_id1,seg_id2,f1,f2,f1_id,f2_id,f1_start,f1_end,f2_start,f2_end,cost
0,224,76094,03December_2011_Saturday_tagesschau_default-8,25July_2010_Sunday_tagesschau_default-5,0,247,96,102,168,174,3.374237
1,224,76084,03December_2011_Saturday_tagesschau_default-8,25July_2010_Sunday_tagesschau_default-5,0,247,96,102,159,171,3.525085
2,413,228032,28May_2011_Saturday_tagesschau_default-2,18June_2010_Friday_tagesschau_default-5,1,748,39,45,21,27,4.630157
3,596,600,28May_2011_Saturday_tagesschau_default-2,28May_2011_Saturday_tagesschau_default-2,1,1,120,126,123,132,5.291626
4,1282,362978,06June_2010_Sunday_tagesschau_default-9,13December_2009_Sunday_tagesschau_default-8,4,1192,15,24,24,30,2.051483
5,1274,1282,06June_2010_Sunday_tagesschau_default-9,06June_2010_Sunday_tagesschau_default-9,4,4,12,18,15,24,2.140472
6,1541,208721,12September_2009_Saturday_tagesschau_default-7,25July_2010_Sunday_tagesschau_default-0,5,687,3,12,48,60,4.621521
7,1540,68630,12September_2009_Saturday_tagesschau_default-7,04December_2011_Sunday_tagesschau_default-0,5,223,3,9,39,51,5.299225
8,1540,392316,12September_2009_Saturday_tagesschau_default-7,18June_2010_Friday_tagesschau_default-9,5,1283,3,9,12,21,5.525269
9,1547,51206,12September_2009_Saturday_tagesschau_default-7,06June_2010_Sunday_tagesschau_default-0,5,168,6,12,51,60,6.345367


In [54]:
knndisc.evaluate()

{'boundary_P': 0.5276,
 'boundary_R': 0.0669,
 'boundary_F': 0.1186,
 'n_clus': 2419,
 'n_node': 3705,
 'grouping_P': 0.4222,
 'grouping_R': 0.3833,
 'grouping_F': 0.4013,
 'token_P': 0.2642,
 'token_R': 0.0297,
 'token_F': 0.0534,
 'type_P': 0.3653,
 'type_R': 0.1356,
 'type_F': 0.1978,
 'coverage': 0.0676,
 'coverageNS': 0.0632,
 'coverageNS_f': 0.0533,
 'ned': 0.4878}

In [40]:
from utils.pipeline import evaluate_discovery, match_pairs_as_clusters, nodes_starts_from1, pickle_save_nodes_clusters

In [308]:
nodes_df, clusters_list = match_pairs_as_clusters(matches_df)
nodes_df, clusters_list = nodes_starts_from1(nodes_df, clusters_list)

postdisc_name = gen_expname(params['clustering'])

postdisc_path = join(params['exp_root'],params['expname'], postdisc_name)
os.makedirs(postdisc_path, exist_ok=True)

pickle_save_nodes_clusters(nodes_df, clusters_list, postdisc_path)  

In [310]:
# grid search

for 


In [311]:


scores = evaluate_discovery(postdisc_path, 
                        jobs=params['njobs'], dataset=params['dataset'],
                        seq_names=seq_names, cnf=params['config_file'])

In [312]:
scores

{'boundary_P': 0.5751,
 'boundary_R': 0.0425,
 'boundary_F': 0.0789,
 'n_clus': 1054,
 'n_node': 1610,
 'grouping_P': 0.4274,
 'grouping_R': 0.3739,
 'grouping_F': 0.3984,
 'token_P': 0.3305,
 'token_R': 0.0187,
 'token_F': 0.0353,
 'type_P': 0.4013,
 'type_R': 0.0863,
 'type_F': 0.1421,
 'coverage': 0.0392,
 'coverageNS': 0.0356,
 'coverageNS_f': 0.0292,
 'ned': 0.4496}

In [21]:
from utils.pipeline_wrappers import try_run_exp
from utils.pipeline import run_matches_discovery, discovery_pipeline
from utils.pipeline_wrappers import gen_expname, run_until_coverage_th, try_run_exp, cv_experiment, monitor_callback, grid_exp
from utils.feature_utils import op_100, get_feats, get_seq_names, get_dh_feats_wrap, load_feats
from utils.helper_fncs import load_json


In [30]:
alg_type = 'knn'
params = load_json(full_path='/home/korhan/Dropbox/config/{}.json'.format(alg_type))

In [31]:
# change desired params here
params['tune']['keys'] = ['a','dim_fix','r', 's','pca']
params['tune']['n_split'] = 1
params['disc']['top_delta'] = 0.05
params['disc']['k'] = 150
params['disc']['r'] = 0.01
params['disc']['s'] = 0.3
params['clustering']['olapthr_m'] = 0.1

# params['disc']['lmin'] = 3
# params['disc']['lmin'] = 6
params['disc']['a'] = 3
params['disc']['pca'] = ''
# params['disc']['use_gpu'] = False


# params['featype'] = 'op100'
params['featype'] = 'c3_right_PCA40'

params_default = params.copy()

# load related features
params['CVset'] = 'A'
seq_names = get_seq_names(params)
feats_dict = load_feats(seq_names, params['featype'], params)
feats_dict[seq_names[0]].shape

(151, 40)

In [29]:
try_run_exp(feats_dict, params)

knn_A_c3_right_PCA40_a3_dim_fix10_emb_typegauss_kernel_k151_lmax28_lmin4_metricL2_normFalse_olapthr_m0.1_pca_seg_typeuniform_top_delta0.05_use_gpuTrue_r0.01_s0.3
0.01
Computing Embeddings
Building index of size 539429x400
Searching index
Selecting good pairs
*** found 290408 matches ***
*** pairwise clustering ***
*** post disc completed, found 3334 segments from 1667 clusters ***
*** Coverage: 1.2800, NED: 33.33
trial 0 th:0.01000 cov:1.28000 err:-8.72000
0.06250000000000001
*** Matches already discovered !!! ***
*** found 290408 matches ***
*** pairwise clustering ***
*** post disc completed, found 19498 segments from 9749 clusters ***
*** Coverage: 5.0700, NED: 41.21
trial 1 th:0.06250 cov:5.07000 err:-4.93000
0.10663324538258578
*** Matches already discovered !!! ***
*** found 290408 matches ***
*** pairwise clustering ***
*** post disc completed, found 34512 segments from 17256 clusters ***
*** Coverage: 7.3800, NED: 48.19
trial 2 th:0.10663 cov:7.38000 err:-2.62000
0.137851307267

{'ned': 50.27,
 'coverage': 8.31,
 'coverageNS': 9.01,
 'coverageNS_f': 11.04,
 'grouping_F': 48.67,
 'grouping_P': 61.93,
 'grouping_R': 40.15,
 'token_F': 5.03,
 'token_P': 18.87,
 'token_R': 2.9,
 'type_F': 18.26,
 'type_P': 39.01,
 'type_R': 11.92,
 'boundary_F': 24.53,
 'boundary_P': 100.0,
 'boundary_R': 14.0,
 'n_clus': 1467,
 'n_node': 2934,
 'exp_path': '/home/korhan/Desktop/tez/knn_exps/knn_A_c3_right_PCA40_a3_dim_fix10_emb_typegauss_kernel_k151_lmax28_lmin4_metricL2_normFalse_olapthr_m0.1_pca_seg_typeuniform_top_delta0.05_use_gpuTrue_r0.01_s0.3/postpairwise_cost0.13785130726793_olap0.1',
 'length_avg': 6.819591693682643,
 'a': 3,
 'dim_fix': 10,
 'emb_type': 'gauss_kernel',
 'k': 151,
 'lmax': 28,
 'lmin': 4,
 'metric': 'L2',
 'norm': False,
 'olapthr_m': 0.1,
 'pca': '',
 'seg_type': 'uniform',
 'top_delta': 0.05,
 'use_gpu': True,
 'r': 0.01,
 's': 0.3}

In [32]:
try_run_exp(feats_dict, params)

knn_A_c3_right_PCA40_a3_dim_fix10_emb_typegauss_kernel_k150_lmax28_lmin4_metricL2_normFalse_olapthr_m0.1_pca_seg_typeuniform_top_delta0.05_use_gpuTrue_r0.01_s0.3
0.01
*** Matches already discovered !!! ***
*** found 300000 matches ***
*** post disc completed, found 3530 segments from 1765 clusters ***
*** Coverage: 1.2900, NED: 33.05
trial 0 th:0.01000 cov:1.29000 err:-8.71000
0.062015503875969005
*** Matches already discovered !!! ***
*** found 300000 matches ***
*** post disc completed, found 20254 segments from 10127 clusters ***
*** Coverage: 5.0800, NED: 40.65
trial 1 th:0.06202 cov:5.08000 err:-4.92000
0.10563167045059421
*** Matches already discovered !!! ***
*** found 300000 matches ***
*** post disc completed, found 35704 segments from 17852 clusters ***
*** Coverage: 7.3500, NED: 47.67
trial 2 th:0.10563 cov:7.35000 err:-2.65000
0.13764247816571562
*** Matches already discovered !!! ***
*** found 300000 matches ***
*** post disc completed, found 47554 segments from 23777 clus

{'ned': 50.92,
 'coverage': 8.78,
 'coverageNS': 9.49,
 'coverageNS_f': 11.55,
 'grouping_F': 49.05,
 'grouping_P': 61.69,
 'grouping_R': 40.77,
 'token_F': 5.38,
 'token_P': 18.95,
 'token_R': 3.13,
 'type_F': 19.73,
 'type_P': 39.51,
 'type_R': 13.15,
 'boundary_F': 25.59,
 'boundary_P': 100.0,
 'boundary_R': 14.69,
 'n_clus': 1635,
 'n_node': 3270,
 'exp_path': '/home/korhan/Desktop/tez/knn_exps/knn_A_c3_right_PCA40_a3_dim_fix10_emb_typegauss_kernel_k150_lmax28_lmin4_metricL2_normFalse_olapthr_m0.1_pca_seg_typeuniform_top_delta0.05_use_gpuTrue_r0.01_s0.3/postpairwise_cost0.15087361202129915_olap0.1',
 'length_avg': 6.818995666387896,
 'a': 3,
 'dim_fix': 10,
 'emb_type': 'gauss_kernel',
 'k': 150,
 'lmax': 28,
 'lmin': 4,
 'metric': 'L2',
 'norm': False,
 'olapthr_m': 0.1,
 'pca': '',
 'seg_type': 'uniform',
 'top_delta': 0.05,
 'use_gpu': True,
 'r': 0.01,
 's': 0.3}

### segment and embed

given `feats_dict{}` , output an array of fixed dim embeddings, also intervals for trace back

In [7]:
%%time
# get segmentation intervals per sequence
intervals_dict = uniform_segmenter(feats_dict, params_seg)
# get embedder
embedder = EmbeddingGaussian(**params_disc)
# do embedding
X, traceback_info = compute_all_embeddings(feats_dict, intervals_dict, embedder)

CPU times: user 4.13 s, sys: 752 ms, total: 4.88 s
Wall time: 4.89 s


In [8]:
X.shape

(518056, 610)

### kNN indexing

given `X: [N x d_fix]` , return  `indices` and `distances` : `[N x k]` 


In [9]:
from utils.knn.faissWrapper import KnnIndex

In [10]:
%%time
k = 100

index = KnnIndex(X, **params_disc)
D, I = index.search(X,k)

CPU times: user 18.2 s, sys: 5.12 s, total: 23.3 s
Wall time: 16.9 s


### matching pairs

find non-overlapping pairs, refine matches